## Answer questions

In [660]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [661]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR, MODELS_DIR, RAW_DATA_DIR

In [662]:
data = pd.read_csv(TRANSFORMED_DATA_DIR / 'transformed_data.csv')

In [663]:
data.shape

(4817, 22)

In [664]:
# Convert date columns to datetime
data['registration_date'] = pd.to_datetime(data['registration_date'])
data['sold_at'] = pd.to_datetime(data['sold_at'])

In [665]:
# Change data types from object to categorical
from src.data import convert_object_columns_to_category

data = convert_object_columns_to_category(data)

In [666]:
from src.data import get_train_test_data

In [658]:
# Read metadata json file from models folder
import json

metadata = json.load(open(MODELS_DIR / 'metadata.json'))

In [659]:
print(f'Model being used is: {metadata["name"]}')

Model being used is: XGBoost linear tuned with RandomizedSearchCV


In [567]:
features = metadata['features']
target = metadata['target']

In [568]:
# Print the features
features

['model_key',
 'mileage',
 'engine_power',
 'fuel',
 'paint_color',
 'car_type',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'age_in_months_when_sold',
 'month_sold_at',
 'season_sold_at',
 'model_initial']

In [569]:
X, y, X_train, X_test, y_train, y_test = get_train_test_data(data, features, target)

In [ ]:
import joblib

In [570]:
# Load the model
model = joblib.load(MODELS_DIR / 'model.pkl')

In [571]:
# Print the model
model

RandomizedSearchCV(cv=4,
                   estimator=XGBRegressor(base_score=None, booster='gblinear',
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=True,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_...
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=None, ...),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'alpha': [0, 0.1, 0.5, 1, 5, 10],
                                        'lambda': [0, 0.1, 0.5, 1, 5, 10],
                                        'n_estimators': [1, 5, 10, 50, 100],
                                        'random_state': [42]},
                   random_state=0, scoring='neg_mean_squared_error', verbose=3)

### Q1

In [572]:
# Read feature importance df from models folder
feature_importance = joblib.load(MODELS_DIR / 'feature_importance_df.pkl')

In [573]:
feature_importance

,feature,importance
6,feature_1,0.186363
13,feature_8,0.171730
8,feature_3,0.121990
11,feature_6,0.121288
9,feature_4,0.110225
7,feature_2,0.058404
5,car_type,0.058048
15,month_sold_at,0.055950
17,model_initial,0.047875
12,feature_7,0.043836


### Q2

As found during the data exploration phase:
- Hybrid and electrical cars are more expensive on average.
- Electrical cars average prices were stable from winter to summer, and were not sold in autumn.
- Diesel and petrol cars had similar average prices, although petrol cars had a drop in average prices starting summer 2018.
- The most expensive car type is, on average, suv, although coupe was most expensive at the start of the year and then dropped below suv also starting in summer.
- Coupe and convertible cars were, on average, more expensive in winter than in summer.
- Vans where more expensive, on average, in spring, summer, and autumn, than in winter.
- Subcompact had generally the lowest average prices.
- Paint color does not seem to generally determine or be associated with the average price, except for color green, which consistently had prices much lower than other colors. Maybe not very popular.
- Orange and white cars were sold for more, on average, during summer than during winter and spring.
- Red cars were the opposite, with lower average prices during summer than during winter and spring.

Find similar observations using estimated price instead of real price

In [574]:
pred = model.predict(X)

In [575]:
data_q2 = data.copy()

In [576]:
data_q2['price'] = pred

In [577]:
from src.plots import plot_avg_target_time_series_by_features

In [578]:
# Load car features
car_features = joblib.load(RAW_DATA_DIR / 'car_features.pkl')

In [579]:
plot_avg_target_time_series_by_features(data_q2, car_features)

In [580]:
# Load small cardinality features
small_cardinality_features = joblib.load(RAW_DATA_DIR / 'small_cardinality_features.pkl')

In [581]:
plot_avg_target_time_series_by_features(data_q2, small_cardinality_features)

In [582]:
from src.questions import ttest_mean_price_difference_between_groups_after_filter

In [583]:
grouping_column = 'season_sold_at'
group_1 = 'winter'
group_2 = 'summer'
for feature in small_cardinality_features:
    for feature_value in data_q2[feature].unique():
        t_stat, p_val = ttest_mean_price_difference_between_groups_after_filter(data_q2, feature, feature_value, grouping_column, group_1, group_2)
        print(f'Feature: {feature}, Feature Value: {feature_value}, T-stat: {t_stat:.4f}, P-value: {p_val:.4f}') 

Feature: fuel, Feature Value: diesel, T-stat: -0.4484, P-value: 0.6539
Feature: fuel, Feature Value: petrol, T-stat: 1.4746, P-value: 0.1446
Feature: fuel, Feature Value: hybrid_petrol, T-stat: nan, P-value: nan
Feature: fuel, Feature Value: electro, T-stat: -0.2886, P-value: 0.8211
Feature: car_type, Feature Value: convertible, T-stat: 1.1477, P-value: 0.2646
Feature: car_type, Feature Value: coupe, T-stat: 2.4129, P-value: 0.0203
Feature: car_type, Feature Value: estate, T-stat: 2.0906, P-value: 0.0369
Feature: car_type, Feature Value: hatchback, T-stat: -0.2854, P-value: 0.7755
Feature: car_type, Feature Value: sedan, T-stat: 4.1597, P-value: 0.0000
Feature: car_type, Feature Value: subcompact, T-stat: 0.7969, P-value: 0.4299
Feature: car_type, Feature Value: suv, T-stat: -1.4214, P-value: 0.1557
Feature: car_type, Feature Value: van, T-stat: -0.9633, P-value: 0.3489
Feature: paint_color, Feature Value: black, T-stat: 0.9927, P-value: 0.3211
Feature: paint_color, Feature Value: grey

### Q3

In [584]:
today_date = '3/1/2024'

In [585]:
data_q3 = data.copy()

In [586]:
# Convert date columns to datetime and mock the sold_at date as today's date
data_q3['registration_date'] = pd.to_datetime(data_q3['registration_date'])
data_q3['sold_at'] = today_date
data_q3['sold_at'] = pd.to_datetime(data_q3['sold_at'])

In [587]:
# Calculate mileage per month
data_q3['mileage_per_month'] = data_q3['mileage'] / data_q3['age_in_months_when_sold']

In [588]:
# Calculate age in month at today date and replace in data_q3
data_q3['age_in_months_when_sold'] = (data_q3['sold_at'].dt.to_period('M') - data_q3['registration_date'].dt.to_period('M')).apply(lambda x: x.n)

In [589]:
# Update estimated mileage at today date
data_q3['mileage'] = data_q3['age_in_months_when_sold'] * data_q3['mileage_per_month']

In [590]:
data_q3[['registration_date', 'sold_at', 'age_in_months_when_sold', 'mileage']].head()

,registration_date,sold_at,age_in_months_when_sold,mileage
0,2012-02-01,2024-03-01,145,286754.859155
1,2016-04-01,2024-03-01,95,60147.954545
2,2012-04-01,2024-03-01,143,374449.585714
3,2014-07-01,2024-03-01,116,345396.744186
4,2014-12-01,2024-03-01,111,269444.175000


In [591]:
X_q3, y_q3, X_train_q3, X_test_q3, y_train_q3, y_test_q3 = get_train_test_data(data_q3, features, target)

In [592]:
# Get estimated prices today
pred_q3 = model.predict(X_q3)

In [593]:
# Add 1 year to the age_in_months_when_sold
X_q3['age_in_months_when_sold'] = X_q3['age_in_months_when_sold'] + 12

In [594]:
# Add 1 year worth of mileage
X_q3['mileage'] = X_q3['mileage'] + 12 * data_q3['mileage_per_month']

In [595]:
# Get estimated prices 1 year later
pred_q3_one_year_later = model.predict(X_q3)

In [596]:
X_q3['price_today'] = pred_q3
X_q3['price_one_year_later'] = pred_q3_one_year_later
X_q3['loss'] = X_q3['price_today'] - X_q3['price_one_year_later'] 

In [597]:
price_today_threshold = 20000
loss_threshold = 2000

In [598]:
# Indentify cars that are candidates for buying
candidate_cars = X_q3[(X_q3['price_today'] >= price_today_threshold) & (X_q3['loss'] <= loss_threshold)]

In [599]:
# Show top 10 candidate cars with lowest loss
candidate_cars.sort_values('loss').head(10)

,model_key,mileage,engine_power,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,...,feature_6,feature_7,feature_8,age_in_months_when_sold,month_sold_at,season_sold_at,model_initial,price_today,price_one_year_later,loss
4346,X5,3090.600000,183,diesel,black,suv,True,True,False,False,...,False,True,True,101,6,summer,X,27768.890625,26601.419922,1167.470703
3910,X5,9278.857143,155,diesel,black,suv,True,True,False,True,...,True,True,True,92,2,winter,X,25818.566406,24620.271484,1198.294922
4121,X6 M,13241.250000,423,petrol,red,suv,True,True,True,False,...,False,True,True,107,4,spring,X,51379.347656,50171.007812,1208.339844
4705,X5 M,18844.800000,230,diesel,silver,suv,True,True,False,True,...,False,False,True,104,8,summer,X,32065.906250,30832.363281,1233.542969
17,650,39463.484375,270,petrol,grey,convertible,True,False,False,False,...,False,True,False,206,9,autumn,6,22476.642578,21238.550781,1238.091797
37,650,39977.079365,270,petrol,grey,convertible,True,False,False,False,...,False,True,False,206,7,summer,6,22196.597656,20957.408203,1239.189453
2980,525,23582.695652,160,diesel,blue,sedan,True,True,True,False,...,True,True,True,106,4,spring,5,22261.832031,21010.177734,1251.654297
4675,X5,41148.046512,160,diesel,grey,suv,True,True,True,False,...,False,True,True,122,8,summer,X,22561.363281,21259.341797,1302.021484
2664,530,37186.666667,195,diesel,grey,sedan,True,True,True,False,...,True,True,True,100,2,winter,5,25628.607422,24311.416016,1317.191406
3718,X5,47620.666667,160,diesel,grey,suv,True,True,True,False,...,False,True,True,122,1,winter,X,21430.183594,20104.884766,1325.298828


In [600]:
# Find the value counts for model_key in candidate_cars if exists. If not try model key initial.
try:
    model_related_value_counts = candidate_cars['model_key'].value_counts()
except KeyError:
    model_related_value_counts = candidate_cars['model_initial'].value_counts()

In [601]:
model_related_value_counts

model_key
X5                  40
X6                  14
640 Gran Coupé       7
X5 M                 6
M3                   5
                    ..
320                  0
318 Gran Turismo     0
630                  0
635                  0
114                  0
Name: count, Length: 75, dtype: int64

In [602]:
# Describe numerical features of candidate cars
candidate_cars.describe()

,mileage,engine_power,age_in_months_when_sold,month_sold_at,price_today,price_one_year_later,loss
count,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000
mean,143407.697505,224.452381,126.230159,5.539683,24462.357422,22811.029297,1651.327393
std,62683.747653,45.460947,18.608885,2.324309,4424.599609,4467.177734,206.349060
min,3090.600000,155.000000,92.000000,1.000000,20040.673828,18250.701172,1167.470703
25%,101960.937500,190.000000,115.000000,4.000000,21609.091309,19906.222656,1509.948730
50%,149272.842767,225.000000,125.000000,6.000000,23087.052734,21430.083984,1665.196289
75%,183052.014706,230.000000,132.750000,8.000000,26030.415039,24508.802246,1803.664062
max,320842.247191,423.000000,206.000000,9.000000,51379.347656,50171.007812,1999.121094


In [603]:
# Show the car with the lowest loss
car_index = candidate_cars.sort_values('loss').index[0]
data_q3.loc[car_index]

maker_key                                  BMW
model_key                                   X5
mileage                                 2723.4
engine_power                               183
registration_date          2016-10-01 00:00:00
fuel                                    diesel
paint_color                              black
car_type                                   suv
feature_1                                 True
feature_2                                 True
feature_3                                False
feature_4                                False
feature_5                                False
feature_6                                False
feature_7                                 True
feature_8                                 True
price                                    49100
sold_at                    2024-03-01 00:00:00
age_in_months_when_sold                     89
month_sold_at                                6
season_sold_at                          summer
model_initial

In [604]:
data.loc[car_index]

maker_key                                  BMW
model_key                                   X5
mileage                                    612
engine_power                               183
registration_date          2016-10-01 00:00:00
fuel                                    diesel
paint_color                              black
car_type                                   suv
feature_1                                 True
feature_2                                 True
feature_3                                False
feature_4                                False
feature_5                                False
feature_6                                False
feature_7                                 True
feature_8                                 True
price                                    49100
sold_at                    2018-06-01 00:00:00
age_in_months_when_sold                     20
month_sold_at                                6
season_sold_at                          summer
model_initial

In [605]:
X_q3.loc[car_index]

model_key                            X5
mileage                          3090.6
engine_power                        183
fuel                             diesel
paint_color                       black
car_type                            suv
feature_1                          True
feature_2                          True
feature_3                         False
feature_4                         False
feature_5                         False
feature_6                         False
feature_7                          True
feature_8                          True
age_in_months_when_sold             101
month_sold_at                         6
season_sold_at                   summer
model_initial                         X
price_today                27768.890625
price_one_year_later       26601.419922
loss                        1167.470703
Name: 4346, dtype: object

In [606]:
# Check cars with the same model_key
data[data['model_key'] == data['model_key'].loc[car_index]].sort_values(
    'price', ascending=False)[['model_key', 'price', 'mileage', 'age_in_months_when_sold']]

,model_key,price,mileage,age_in_months_when_sold
4588,X5,55700,24912,22
3993,X5,52200,33639,21
4645,X5,50600,80307,54
4525,X5,50000,82397,39
4346,X5,49100,612,20
...,...,...,...,...
4423,X5,5100,249546,171
4607,X5,4600,217045,169
4678,X5,3600,220242,180
4347,X5,3500,350615,205


### Q3 assumming today is a month after latest sold at date in the data

In [607]:
today_date = data['sold_at'].max()

In [608]:
data_q3 = data.copy()

In [609]:
# Convert date columns to datetime and mock the sold_at date as today's date
data_q3['registration_date'] = pd.to_datetime(data_q3['registration_date'])
data_q3['sold_at'] = today_date
data_q3['sold_at'] = pd.to_datetime(data_q3['sold_at'])

In [610]:
# Calculate mileage per month
data_q3['mileage_per_month'] = data_q3['mileage'] / data_q3['age_in_months_when_sold']

In [611]:
# Calculate age in month at today date and replace in data_q3
data_q3['age_in_months_when_sold'] = (data_q3['sold_at'].dt.to_period('M') - data_q3['registration_date'].dt.to_period('M')).apply(lambda x: x.n)

In [612]:
# Update estimated mileage at today date
data_q3['mileage'] = data_q3['age_in_months_when_sold'] * data_q3['mileage_per_month']

In [613]:
data_q3[['registration_date', 'sold_at', 'age_in_months_when_sold', 'mileage']].head()

,registration_date,sold_at,age_in_months_when_sold,mileage
0,2012-02-01,2018-09-01,79,156231.957746
1,2016-04-01,2018-09-01,29,18360.954545
2,2012-04-01,2018-09-01,77,201626.700000
3,2014-07-01,2018-09-01,50,148877.906977
4,2014-12-01,2018-09-01,45,109234.125000


In [614]:
X_q3, y_q3, X_train_q3, X_test_q3, y_train_q3, y_test_q3 = get_train_test_data(data_q3, features, target)

In [615]:
# Get estimated prices today
pred_q3 = model.predict(X_q3)

In [616]:
# Add 1 year to the age_in_months_when_sold
X_q3['age_in_months_when_sold'] = X_q3['age_in_months_when_sold'] + 12

In [617]:
# Add 1 year worth of mileage
X_q3['mileage'] = X_q3['mileage'] + 12 * data_q3['mileage_per_month']

In [618]:
# Get estimated prices 1 year later
pred_q3_one_year_later = model.predict(X_q3)

In [619]:
X_q3['price_today'] = pred_q3
X_q3['price_one_year_later'] = pred_q3_one_year_later
X_q3['loss'] = X_q3['price_today'] - X_q3['price_one_year_later'] 

In [620]:
# Indentify cars that are candidates for buying
candidate_cars = X_q3[(X_q3['price_today'] >= price_today_threshold) & (X_q3['loss'] <= loss_threshold)]

In [621]:
# Show top 10 candidate cars with lowest loss
candidate_cars.sort_values('loss').head(10)

,model_key,mileage,engine_power,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,...,feature_6,feature_7,feature_8,age_in_months_when_sold,month_sold_at,season_sold_at,model_initial,price_today,price_one_year_later,loss
2400,420 Gran Coupé,597.333333,120,diesel,blue,hatchback,True,True,False,False,...,True,True,True,64,8,summer,4,22060.671875,20902.531250,1158.140625
4346,X5,1071.000000,183,diesel,black,suv,True,True,False,False,...,False,True,True,35,6,summer,X,34189.972656,33022.503906,1167.468750
3910,X5,2622.285714,155,diesel,black,suv,True,True,False,True,...,True,True,True,26,2,winter,X,32409.187500,31210.894531,1198.292969
4121,X6 M,5073.750000,423,petrol,red,suv,True,True,True,False,...,False,True,True,41,4,spring,X,58025.210938,56816.871094,1208.339844
4705,X5 M,6885.600000,230,diesel,silver,suv,True,True,False,True,...,False,False,True,38,8,summer,X,38850.398438,37616.851562,1233.546875
17,650,26819.843750,270,petrol,grey,convertible,True,False,False,False,...,False,True,False,140,9,autumn,6,29286.158203,28048.064453,1238.093750
37,650,27168.888889,270,petrol,grey,convertible,True,False,False,False,...,False,True,False,140,7,summer,6,29012.128906,27772.945312,1239.183594
4353,X3,15729.389831,120,diesel,brown,suv,True,True,True,True,...,False,True,True,74,6,summer,X,23920.218750,22672.917969,1247.300781
2980,525,8899.130435,160,diesel,blue,sedan,True,True,True,False,...,True,True,True,40,4,spring,5,29145.929688,27894.275391,1251.654297
3667,i3,10796.857143,75,hybrid_petrol,black,subcompact,True,True,True,False,...,False,True,True,46,3,spring,i,21983.613281,20726.591797,1257.021484


In [622]:
# Find the value counts for model_key in candidate_cars if exists. If not try model key initial.
try:
    model_related_value_counts = candidate_cars['model_key'].value_counts()
except KeyError:
    model_related_value_counts = candidate_cars['model_initial'].value_counts()

In [623]:
model_related_value_counts

model_key
X5                  92
X3                  86
530                 26
520                 25
X6                  25
                    ..
335 Gran Turismo     0
418 Gran Coupé       0
630                  0
116                  0
114                  0
Name: count, Length: 75, dtype: int64

In [624]:
# Describe numerical features of candidate cars
candidate_cars.describe()

,mileage,engine_power,age_in_months_when_sold,month_sold_at,price_today,price_one_year_later,loss
count,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000,491.000000
mean,90831.308689,175.364562,68.743381,5.378819,26136.613281,24411.625000,1724.985596
std,42375.784171,46.768334,21.598359,2.131178,5387.859863,5415.497559,187.660980
min,597.333333,75.000000,26.000000,1.000000,20025.001953,18114.406250,1158.140625
25%,61179.985642,135.000000,55.000000,4.000000,21831.587891,20117.350586,1612.440430
50%,87819.600000,173.000000,65.000000,5.000000,24662.115234,22886.771484,1753.015625
75%,117064.735775,190.000000,78.000000,7.000000,29405.572266,27903.879883,1867.000977
max,239292.452830,423.000000,140.000000,9.000000,58025.210938,56816.871094,1999.515625


In [625]:
# Show the car with the lowest loss
car_index = candidate_cars.sort_values('loss').index[0]
data_q3.loc[car_index]

maker_key                                  BMW
model_key                       420 Gran Coupé
mileage                             485.333333
engine_power                               120
registration_date          2014-05-01 00:00:00
fuel                                    diesel
paint_color                               blue
car_type                             hatchback
feature_1                                 True
feature_2                                 True
feature_3                                False
feature_4                                False
feature_5                                False
feature_6                                 True
feature_7                                 True
feature_8                                 True
price                                    30300
sold_at                    2018-09-01 00:00:00
age_in_months_when_sold                     52
month_sold_at                                8
season_sold_at                          summer
model_initial

In [626]:
data.loc[car_index]

maker_key                                  BMW
model_key                       420 Gran Coupé
mileage                                    476
engine_power                               120
registration_date          2014-05-01 00:00:00
fuel                                    diesel
paint_color                               blue
car_type                             hatchback
feature_1                                 True
feature_2                                 True
feature_3                                False
feature_4                                False
feature_5                                False
feature_6                                 True
feature_7                                 True
feature_8                                 True
price                                    30300
sold_at                    2018-08-01 00:00:00
age_in_months_when_sold                     51
month_sold_at                                8
season_sold_at                          summer
model_initial

In [627]:
X_q3.loc[car_index]

model_key                  420 Gran Coupé
mileage                        597.333333
engine_power                          120
fuel                               diesel
paint_color                          blue
car_type                        hatchback
feature_1                            True
feature_2                            True
feature_3                           False
feature_4                           False
feature_5                           False
feature_6                            True
feature_7                            True
feature_8                            True
age_in_months_when_sold                64
month_sold_at                           8
season_sold_at                     summer
model_initial                           4
price_today                  22060.671875
price_one_year_later          20902.53125
loss                          1158.140625
Name: 2400, dtype: object

In [628]:
# Check cars with the same model_key
data[data['model_key'] == data['model_key'].loc[car_index]].sort_values(
    'price', ascending=False)[['model_key', 'price', 'mileage', 'age_in_months_when_sold']]

,model_key,price,mileage,age_in_months_when_sold
2170,420 Gran Coupé,32100,27547,28
2400,420 Gran Coupé,30300,476,51
1806,420 Gran Coupé,30200,104133,34
2138,420 Gran Coupé,29800,64380,30
2222,420 Gran Coupé,29500,40168,39
1904,420 Gran Coupé,29500,27966,22
3403,420 Gran Coupé,29300,54836,52
2422,420 Gran Coupé,29200,77358,41
2395,420 Gran Coupé,28100,64500,45
1978,420 Gran Coupé,27400,139258,41


### Q4

In [630]:
print(f'Test MSE: {metadata["mse_test"]:.2f}')
print(f'Test RMSE: {metadata["rmse_test"]:.2f}')
print(f'Test MAE: {metadata["mae_test"]:.2f}')
print(f'Test R2: {metadata["r2_test"]:.2f}')

Test MSE: 17140475.03
Test RMSE: 4140.11
Test MAE: 2858.92
Test R2: 0.74


### Q5

Add other findings from data exploration